In [1]:
import os
import pandas as pd
import re
import math
import duckdb
import numpy as np
# Display all columns
pd.set_option('display.max_columns', None)

# Set max column width to a large value
pd.set_option('display.max_colwidth', None)

# Create a new DuckDB database file
con = duckdb.connect('balancer_dynamic_fee.duckdb')

In [2]:
# Read the CSV file line by line
file_path = 'balweth_decoded.csv'
lines = []

with open(file_path, 'r') as file:
    lines = file.readlines()

# Initialize dictionaries to store lines for each transaction type
transaction_dict = {}

# Function to parse fields without combining text between single quotes
def parse_line(line):
    fields = line.split(',')
    fields = [field.strip() for field in fields]
    return fields

# Iterate over each line to categorize it based on the 6th field
for line in lines:
    fields = parse_line(line.strip())
    if len(fields) > 5:  # Ensure there are at least 6 fields after cleaning
        transaction_type = fields[5]
        if transaction_type not in transaction_dict:
            transaction_dict[transaction_type] = []
        transaction_dict[transaction_type].append(fields)

# Create separate DataFrames for each transaction type
dataframes = {}
for transaction_type, data in transaction_dict.items():
    # Determine the number of columns for this transaction type
    num_columns = max(len(fields) for fields in data)
    # Pad the data with empty strings to ensure all rows have the same number of columns
    padded_data = [fields + [''] * (num_columns - len(fields)) for fields in data]
    # Create DataFrame
    df = pd.DataFrame(padded_data, columns=[f'Field_{i+1}' for i in range(num_columns)])
    dataframes[transaction_type] = df

# Print out the names of the created DataFrames
for transaction_type in dataframes.keys():
    print(f'Created DataFrame: {transaction_type}')

SwapFeePercentageChanged = dataframes['SwapFeePercentageChanged']


Created DataFrame: Transfer
Created DataFrame: SwapFeePercentageChanged
Created DataFrame: Approval


In [3]:
SwapFeePercentageChanged

,Field_1,Field_2,Field_3,Field_4,Field_5,Field_6,Field_7,Field_8,Field_9,Field_10,Field_11,Field_12,Field_13,Field_14,Field_15,Field_16
0,12505422,124,111,0x0e5c53950574cb933da47d5a885ae7f134849f43fa77eb818d9efd5cc65dbed0,0x5c6Ee304399DBdB9C8Ef030aB642B10820DB8F56,SwapFeePercentageChanged,1800000000000000,None,None,None,None,None,None,None,None,None
1,12654826,142,189,0x7aadce84f11aa371f3ae09e56fbd46db03da1d3723d6df345fbde0f9a71548c1,0x5c6Ee304399DBdB9C8Ef030aB642B10820DB8F56,SwapFeePercentageChanged,1400000000000000,None,None,None,None,None,None,None,None,None
2,12693310,154,301,0xd2bb72c0cfcea82cd4f7c3ff02fe7fe5b52bae983a71dfa0b746829d3233c0d5,0x5c6Ee304399DBdB9C8Ef030aB642B10820DB8F56,SwapFeePercentageChanged,1100000000000000,None,None,None,None,None,None,None,None,None
3,12738108,124,251,0x6afe74f5bd6147a6a44b309cb60b30e83d0ae6957517255240b9c80004419d68,0x5c6Ee304399DBdB9C8Ef030aB642B10820DB8F56,SwapFeePercentageChanged,900000000000000,None,None,None,None,None,None,None,None,None
4,12775310,135,288,0xbc08290f656da4af562dcd64eef20e980972d2e554a6fa97439a49374eae38cc,0x5c6Ee304399DBdB9C8Ef030aB642B10820DB8F56,SwapFeePercentageChanged,1300000000000000,None,None,None,None,None,None,None,None,None
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6001,20000486,24,228,0xa33a84df75a78b325cb7c1686997d7710ebb5f4aea547e0f308ff1bdd80e248c,0x5c6Ee304399DBdB9C8Ef030aB642B10820DB8F56,SwapFeePercentageChanged,10000000000000000,None,None,None,None,None,None,None,None,None
6002,20000964,21,142,0x2702be8907a9a2a6b73b301f4482056889c36b49615def510b69993eabc72084,0x5c6Ee304399DBdB9C8Ef030aB642B10820DB8F56,SwapFeePercentageChanged,2000000000000000,None,None,None,None,None,None,None,None,None
6003,20000964,21,149,0x2702be8907a9a2a6b73b301f4482056889c36b49615def510b69993eabc72084,0x5c6Ee304399DBdB9C8Ef030aB642B10820DB8F56,SwapFeePercentageChanged,10000000000000000,None,None,None,None,None,None,None,None,None
6004,20000984,21,138,0x98530576986ee76f954f6185fab934b445802a0a0019164fdfbdd134e17121b7,0x5c6Ee304399DBdB9C8Ef030aB642B10820DB8F56,SwapFeePercentageChanged,2000000000000000,None,None,None,None,None,None,None,None,None


In [4]:
selected_columns = {'Field_1': 'block_number', 'Field_3': 'log_index', 'Field_4': 'transaction_hash', 'Field_6':'event', 'Field_7': 'fee'}
SwapFeePercentageChanged = SwapFeePercentageChanged[list(selected_columns.keys())].rename(columns=selected_columns)

In [5]:
SwapFeePercentageChanged

,block_number,log_index,transaction_hash,event,fee
0,12505422,111,0x0e5c53950574cb933da47d5a885ae7f134849f43fa77eb818d9efd5cc65dbed0,SwapFeePercentageChanged,1800000000000000
1,12654826,189,0x7aadce84f11aa371f3ae09e56fbd46db03da1d3723d6df345fbde0f9a71548c1,SwapFeePercentageChanged,1400000000000000
2,12693310,301,0xd2bb72c0cfcea82cd4f7c3ff02fe7fe5b52bae983a71dfa0b746829d3233c0d5,SwapFeePercentageChanged,1100000000000000
3,12738108,251,0x6afe74f5bd6147a6a44b309cb60b30e83d0ae6957517255240b9c80004419d68,SwapFeePercentageChanged,900000000000000
4,12775310,288,0xbc08290f656da4af562dcd64eef20e980972d2e554a6fa97439a49374eae38cc,SwapFeePercentageChanged,1300000000000000
...,...,...,...,...,...
6001,20000486,228,0xa33a84df75a78b325cb7c1686997d7710ebb5f4aea547e0f308ff1bdd80e248c,SwapFeePercentageChanged,10000000000000000
6002,20000964,142,0x2702be8907a9a2a6b73b301f4482056889c36b49615def510b69993eabc72084,SwapFeePercentageChanged,2000000000000000
6003,20000964,149,0x2702be8907a9a2a6b73b301f4482056889c36b49615def510b69993eabc72084,SwapFeePercentageChanged,10000000000000000
6004,20000984,138,0x98530576986ee76f954f6185fab934b445802a0a0019164fdfbdd134e17121b7,SwapFeePercentageChanged,2000000000000000


In [6]:
con.execute('CREATE TABLE balweth_fee_change AS SELECT * FROM SwapFeePercentageChanged').fetchdf()

# Commit the changes
con.commit()

# Close the connection
con.close()